<a href="https://colab.research.google.com/github/filispeen/Meshroom-colab.ipynb/blob/main/Meshroom_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
* **`alicevision_version`**: Version alicevion/meshroom used to work with photos

---

* **`upload_type`** : Select the type of upload method.

* **`photo_archive_path`** : Path to your zip file relative to the root of your Google Drive if google_drive is selected. Must state the file and its extension (.zip) unless gdrive_relative is selected.

* **`url_photo_zip_url`** : Specify the URL to the blend/zip file if url is selected.

---

* **`download_type`** : Select the type of download method. **`gdrive_direct`** enables the archive to be outputted directly to Google Drive.

* **`drive_output_path`** : Path to your zip file in Google Drive.

After you are done, go to **Runtime > Run All (Ctrl + F9)** and upload your files or have Google Drive authorised below. See the GitHub repo for more information.

In [ ]:
alicevision_version = "2021.1.0" #@param ["2021.1.0", "2020.1.1", "2019.2.0", "2019.1.0", "2018.1.0"]
#@markdown ---
upload_type = "google_drive" #@param ["direct", "google_drive", "url"]
photo_archive_path = "Meshroom/\u003Czip-only>.zip" #@param {type:"string"}
url_photo_zip_url = "https://site.com/file.zip" #@param {type:"string"}
#@markdown ---
download_type = "google_drive" #@param ["direct", "google_drive"]
drive_output_path = "meshroomoutput" #@param {type:"string"}



In [ ]:
#@title Checking GPU { vertical-output: true, display-mode: "form" }
import os

!ls

!LD_LIBRARY_PATH=/usr/lib/nvidia:$LD_LIBRARY_PATH

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

In [ ]:
#@title Downloading archive { vertical-output: true, display-mode: "form" }
import shutil
import os
from google.colab import files, drive

!apt install unzip

uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -O content.zip -nc $url_photo_zip_url
    uploaded_filename = os.path.basename(url_photo_zip_url)
elif upload_type == 'google_drive':
    !rm -r content.zip
    shutil.copy('/drive/My Drive/' + photo_archive_path, ".")
    !mv *.zip content.zip 
    uploaded_filename = os.path.basename(photo_archive_path)

In [ ]:
#@title Downloading meshroom { vertical-output: true, display-mode: "form" }
alicevision_url_dict = {'2021.1.0' : "https://github.com/alicevision/Meshroom/releases/download/v2021.1.0/Meshroom-2021.1.0-linux-cuda10.tar.gz",
                        '2020.1.1' : "https://github.com/alicevision/Meshroom/releases/download/v2020.1.1/Meshroom-2020.1.0-linux-cuda10.tar.gz",
                        '2019.2.0' : "https://github.com/alicevision/Meshroom/releases/download/v2019.2.0/Meshroom-2019.2.0-linux.tar.gz",
                        '2019.1.0' : "https://github.com/alicevision/Meshroom/releases/download/v2019.1.0/Meshroom-2019.1.0-linux.tar.gz",
                        '2018.1.0' : "https://github.com/alicevision/Meshroom/releases/download/v2018.1.0/Meshroom-2018.1.0-linux.tar.gz"}

alicevision_url = alicevision_url_dict[alicevision_version]
base_url = os.path.basename(alicevision_url)

!rm -r $alicevision_version
!mkdir $alicevision_version
!wget -nc $alicevision_url
!tar -xkvf $base_url -C ./$alicevision_version --strip-components=1

In [ ]:
#@title Meshroom starting work { vertical-output: true, display-mode: "form" }
%cd

%cd ../content

!rm -r $alicevision_version/content/*

!unzip content.zip -d $alicevision_version/content

%cd $alicevision_version/

!ls

!rm -r output

!mkdir output

!./meshroom_batch --input content --output output

#!bash aliceVision_cameraInit-2.0 --imageFolder "./content/content/" --sensorDatabase "./content/content/cameraSensors.db" --defaultFieldOfView 45.0 --groupCameraFallback folder --allowedCameraModels pinhole,radial1,radial3,brown,fisheye4,fisheye1 --useInternalWhiteBalance True --viewIdMethod metadata --verboseLevel info --output "./content/content/cameraInit.sfm" --allowSingleView 1 --input "./content/content//viewpoints.sfm"

#!bash aliceVision_featureExtraction  --input "./content/content/cameraInit.sfm" --describerTypes sift --describerPreset normal --describerQuality normal --contrastFiltering GridSort --gridFiltering True --forceCpuExtraction True --maxThreads 0 --verboseLevel info --output "./content/content/FeatureExtraction" --rangeStart 0 --rangeSize 0

#!bash aliceVision_featureExtraction  --input "./content/content/cameraInit.sfm" --describerTypes sift --describerPreset normal --describerQuality normal --contrastFiltering GridSort --gridFiltering True --forceCpuExtraction True --maxThreads 0 --verboseLevel info --output "./content/content/FeatureExtraction" --rangeStart 0 --rangeSize 40

#!bash aliceVision_featureExtraction  --input "./content/content/cameraInit.sfm" --describerTypes sift --describerPreset normal --describerQuality normal --contrastFiltering GridSort --gridFiltering True --forceCpuExtraction True --maxThreads 0 --verboseLevel info --output "./content/content/FeatureExtraction" --rangeStart 0 --rangeSize 80

#!bash aliceVision_imageMatching  --input "./content/content/cameraInit.sfm" --featuresFolders "./content/content/FeatureExtraction" --method VocabularyTree --tree "./Meshroom-2021.1.0-linux-cuda10/aliceVision/share/aliceVision/vlfeat_K80L3.SIFT.tree" --weights "" --minNbImages 200 --maxDescriptors 500 --nbMatches 50 --verboseLevel info --output "./content/content/imageMatches.txt"

#!bash aliceVision_imageMatching  --input "./content/content/cameraInit.sfm" --featuresFolders "./content/content/FeatureExtraction" --method VocabularyTree --tree "./Meshroom-2021.1.0-linux-cuda10/aliceVision/share/aliceVision/vlfeat_K80L3.SIFT.tree" --weights "" --minNbImages 200 --maxDescriptors 500 --nbMatches 50 --verboseLevel info --output "./content/content/imageMatches.txt"

#!bash aliceVision_featureMatching-2.0  --input "./content/content/cameraInit.sfm" --featuresFolders "./content/content/" --imagePairsList "./content/content/imageMatches.txt" --describerTypes sift --photometricMatchingMethod ANN_L2 --geometricEstimator acransac --geometricFilterType fundamental_matrix --distanceRatio 0.8 --maxIteration 2048 --geometricError 0.0 --knownPosesGeometricErrorMax 5.0 --maxMatches 0 --savePutativeMatches False --crossMatching False --guidedMatching False --matchFromKnownCameraPoses False --exportDebugFiles False --verboseLevel info --output "./content/" --rangeStart 0 --rangeSize 20


In [ ]:
#@title Making archive and importing on google drive/making download link for you { vertical-output: true, display-mode: "form" }
output_folder_name = "output"

if download_type == 'google_drive':
  shutil.make_archive(output_folder_name, 'zip', 'output')
  shutil.copy(output_folder_name + '.zip', '/drive/My Drive/' + drive_output_path)

if download_type == 'direct':
  shutil.make_archive(output_folder_name, 'zip', 'output')
  files.download(output_folder_name + '.zip')

raise SystemExit("See ya later! Script by @FILISPEEN")




# Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

# License
```
MIT License

Copyright (c) 2020-2022 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```





